# Politische Reden als Datenquelle

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import re
import json

from studienarbeit.utils.cleaning import Cleaning
from studienarbeit.utils.plots import Plots
from studienarbeit.utils.sentiment import Sentiment
from studienarbeit.utils.split_text import SplitText
from functools import reduce
from pathlib import Path
from dotenv import load_dotenv
from collections import Counter
from nltk import ngrams
from nltk.tokenize import sent_tokenize
from tqdm import tqdm

In [ ]:
load_dotenv()
sns.set(style="white", palette="muted", rc={"figure.figsize": (20, 8)})
tqdm.pandas()

In [ ]:
data_dir = Path("../../data/speeches")

In [ ]:
with open("../../data/party_colors.json", "r") as f:
  party_palette = json.load(f)
plots = Plots(party_palette)
sentiment = Sentiment()
cleaning = Cleaning()
split_text = SplitText()

## Business Understanding

Im Folgenden wird der [Datensatz bestehend aus Bundestagsreden](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/FIKIBO), bereitgestellt von [Open Discourse](https://opendiscourse.de/) initial untersucht. Politische Reden, in diesem Fall aus dem deutschen Bundestag, sollen als ein Standbein als Trainingsdaten für das Klassifikationsproblem genutzt werden. Der Datensatz bietet den Vorteil, dass eine große Menge an gelabelten Texten zur Verfügung steht sowie alle möglichen Themenbereiche abdeckt, die im Laufe der Jahre im Bundestag debattiert wurden. Da die Reden deutlich länger sind als Tweets (siehe unten), muss eine Lösung gefunden werden, wie die Redetexte aufgeteilt werden können, um sie dem Modell als Input zur Verfügung zu stellen. Andererseits kann dies auche als Vorteil gesehen werden, da die Reden deutlich mehr Informationen enthalten als Tweets und somit dem Modell eine bessere Chance bieten, auch aufgrund inhaltlichen Zusammenhängen zu entscheiden.

## Data Understanding

### `factions`


In [ ]:
df_factions = pd.read_feather(data_dir / "src" / "factions.feather")

In [ ]:
df_factions

Die `factions`-Datei verknüpft die ID der Parteien mit deren Namen. Für die für uns relevanten Parteien sollten wir uns frühzeitig auf ein einheitliches Mapping zwischen Label und Partei einigen, um spätere Probleme zu vermeiden.


### `politicians`


In [ ]:
df_politicians = pd.read_feather(data_dir / "src" / "politicians.feather")

In [ ]:
df_politicians

Die `politicians`-Datei verknüpft die ID der Politiker mit deren Namen inkl. weiteren Informationen. Da wir zunächst nur nach Parteien klassifizieren wollen, ist diese Datei für uns nicht relevant.


### `electoral_terms`


In [ ]:
df_electoral_terms = pd.read_feather(data_dir / "src" / "electoral_terms.feather")

In [ ]:
df_electoral_terms

Die `electoral_terms`-Datei verknüpft die ID der Wahlperioden mit derem Start- und Enddatum. Um in allen Datenquellen vergleichbare Themen, Sprache und Parteien zu haben, sollten alle Daten auf einen ähnlichen Zeitraum beschränkt werden. Da sich die uns zur Verfügung stehenden Tweet Daten auf die Bundestagswahl 2021 und die Wahlperiode 19 beziehen, sind vor allem Reden als der 19. Wahlperiode relevant.


### `contributions`


In [ ]:
df_contributions = pd.read_feather(data_dir / "src" / "contributions_extended.feather")

In [ ]:
df_contributions

Die `contributions`-Datei enthält Zwischenrufe oder sonstige Reaktionen im Plenum, die keine richtigen Reden sind. Da diese meist nur sehr kurz und nicht wirklich aussagekräftig sind, werden sie für uns nicht relevant sein.

### `speeches`


In [ ]:
df_speeches = pd.read_feather(data_dir / "src" / "speeches.feather", columns=["electoralTerm", "speechContent", "politicianId", "factionId"])

In [ ]:
df_speeches

Wie gerade angemerkt, sind für uns erstmal die Reden aus der 19. Wahlperiode relevant:


In [ ]:
df_speeches = df_speeches[df_speeches["electoralTerm"] == 19].reset_index(drop=True)
df_speeches

Aus dieser Wahlperiode bleiben zunächst etwa 61.000 Reden übrig. Davon sind jedoch noch jene abzuziehen, die keiner Partei zugeordnet sind, weil beispielsweise das Präsidium des Bundestage gesprochen hat. Zudem wird mithilfe der `factionId` der Namen der Fraktion bestimmt.


In [ ]:
factions_of_interest = [0, 3, 4, 6, 13, 23]
df_speeches = df_speeches[df_speeches["factionId"].isin(factions_of_interest)]

party_mapping = {
    0: "AfD",
    3: "Grüne",
    4: "Union",
    6: "Linke",
    13: "FDP",
    23: "SPD"
}
df_speeches["party"] = df_speeches["factionId"].map(party_mapping)
df_speeches = df_speeches.drop(columns=["factionId"])

df_speeches

Somit reduziert sich die Anzahl an verbleibender Reden auf knapp 29.000.


Da der Wert von `electoralTerm` nun konstant ist, kann die Spalte gelöscht werden:

In [ ]:
df_speeches = df_speeches.drop(columns=["electoralTerm"])
df_speeches

#### `Anzahl an Reden pro Fraktion`

Wenn man sich die Anzahl an Reden pro Fraktion anschaut, fällt auf, dass die Fraktionen der CDU/CSU und der SPD deutlich mehr Reden haben als die anderen Parteien, was auch zu erwarten ist. Im Zuge der Data Preparation muss hierzu sichergestellt werden, dass ein Ungleichgewicht vermieden wird.


In [ ]:
plots.party_count(df_speeches, title="Anzahl an Reden nach Partei")

### `Anzahl an Rednern pro Fraktion`

In [ ]:
plots.politician_count(df_speeches, title="Anzahl an Politikern nach Partei")

### `Häufige 3-grams`

In [ ]:
for party in ["AfD", "FDP", "Grüne", "Linke", "SPD", "Union"]:
    df_current_party = df_speeches[df_speeches["party"] == party]
    ngram_counts = Counter(ngrams(" ".join(df_current_party["speechContent"].tolist()).split(), 3))
    print(f"{party}: " + str(ngram_counts.most_common(5)))
    print("\n")

Auffallend sind Formulierung zur Begrüßung zu Beginn der Rede, die häufig vorkommen. Diese können später entfernt werden.

### `Verteilung der Anzahl an...`

In [ ]:
df_speeches["char_count"] = df_speeches["speechContent"].progress_apply(len).astype("int16")
df_speeches["word_count"] = df_speeches["speechContent"].progress_apply(lambda x : len(x.split())).astype("int16")
df_speeches["sentence_count"] = df_speeches["speechContent"].progress_apply(lambda x : len(sent_tokenize(x))).astype("int16")

#### `Zeichen`

In [ ]:
plots.text_count(df_speeches, "char_count", "Anzahl an Zeichen in Reden nach Partei", "Anzahl Zeichen", 10000)

#### `Wörtern`

In [ ]:
plots.text_count(df_speeches, "word_count", "Anzahl an Wörtern in Reden nach Partei", "Anzahl Wörter", 1300)

#### `Sätzen`

In [ ]:
plots.text_count(df_speeches, "sentence_count", "Anzahl an Sätzen in Reden nach Partei", "Anzahl Sätze", 100)

### `Sonderzeichen`

In [ ]:
reduce(lambda a, b: set((*a, *b)), df_speeches["speechContent"].apply(np.array))

Schaut man sich an, welche Zeichen in den Reden vorkommen, fällt auf, dass neben dem regulären Alphabet auch einige unerwünschte Sonderzeichen enthalten sind, die im Zuge der Data Preparation entfernt werden müssen.


## Data Preparation

In [ ]:
df_prep = df_speeches.copy().reset_index(drop=True)

In [ ]:
df_prep = df_prep[df_prep["politicianId"] != -1]
df_prep

In [ ]:
df_prep = df_prep[df_prep["word_count"] > 100].reset_index(drop=True)
df_prep

In [ ]:
df_prep = split_text.split_dataframe_texts(df_prep, "speechContent", 512)
df_prep

In [ ]:
def initial_cleaning(text):
    text = re.sub("[\u2022\u2023\u25E6\u2043\u2219\uf0b7\u25fc]\s", " ", text)
    text = re.sub("({\d*})", "", text)
    text = re.sub("\(\w*\)", "", text)
    text = text.replace(". –", ". ")
    
    text = re.sub("\n", " ", text)
    text = re.sub("\t", " ", text)
    text = re.sub("\s+", " ", text)
    text = text.strip()
    
    return text

In [ ]:
df_prep["clean_text"] = df_prep["speechContent"].progress_apply(lambda x: cleaning.clean_text(initial_cleaning(x), keep_punctuation=True, keep_upper=True)).astype("string[pyarrow]")
df_prep = df_prep.drop(columns=["speechContent", "politicianId"])

df_prep["tokenized_text"] = df_prep["clean_text"].progress_apply(lambda x: cleaning.filter_text(cleaning.stemm_text(x))).astype("string[pyarrow]")

if (data_dir / "cache").exists() == False:
    (data_dir / "cache").mkdir()

df_prep = df_prep.reset_index()
df_prep.to_feather(data_dir / "cache/speeches_prep.feather")

In [ ]:
df_prep

In [ ]:
df_final = df_prep.copy().reset_index(drop=True)

In [ ]:
with open("../../data/party_encoding.json", "r") as f:
  party_encoding = json.load(f)
  df_final["party"] = df_final["party"].map(party_encoding)

In [ ]:
df_final

In [ ]:
df_final.to_parquet(data_dir / "speeches.parquet", index=False)